# 7. Feature engineering

## Setup and imports

In [5]:
from functions import *
from statsmodels.tsa.stattools import acf, pacf
academic_style()

## Load the data

In [6]:
df = load_data()
df = df.sort_index()
demand = df["demand"].asfreq("h").to_frame(name="demand")
demand = demand.interpolate(method="time", limit_direction="both")
demand = demand.reset_index().rename(columns={"index": "timestamp"})

print(
    f"Observations: {len(demand):,}"
)

Observations: 8,760


## 1. Analyze stationarity

In [7]:
stationarity_table = stationarity_tests(demand["demand"], lags=30)
display(stationarity_table)

C:\Users\danyi\Desktop\HomeEnergyManagementSystem\functions.py:242: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




,test,statistic,p_value,lags,critical_5%,interpretation
0,ADF,-11.7734,0.00,30,-2.8619,Stationary
1,KPSS,1.2719,0.01,30,0.4630,Non-stationary


## 2. Next ACF and PACF plots are constructed.

In [8]:
nlags = 72
acf_vals = acf(demand["demand"], nlags=nlags)
pacf_vals = pacf(demand["demand"], nlags=nlags)

acf_df = pd.DataFrame({"lag": range(len(acf_vals)), "value": acf_vals})
pacf_df = pd.DataFrame({"lag": range(len(pacf_vals)), "value": pacf_vals})

fig_acf_pacf = plot_acf_pacf(
    acf_df,
    pacf_df,
    title="Hourly demand (ACF and PACF)",
)

save_fig_plotly(fig_acf_pacf, "ex7_fig1_stats_acf_pacf.svg", width=1200, height=500)

fig_acf_pacf.show()

## 3. Use ARIMA as autoregressive model and SARIMA to see the influence of the seasonal differences.

In [9]:
FORECAST_HORIZON = 24
VALIDATION_DAYS = 7

cutoff = demand["timestamp"].max() - pd.Timedelta(days=VALIDATION_DAYS)

train = demand.loc[demand["timestamp"] < cutoff].set_index("timestamp")["demand"].asfreq("h")
test = demand.loc[demand["timestamp"] >= cutoff].set_index("timestamp")["demand"][:FORECAST_HORIZON]

# 3(a) ARIMA(p,1,q) grid search

In [10]:
p_values = [1, 2, 3]
q_values = [1, 2, 3, 4]

grid_df, best_p, best_q = gridsearch_arima(train, test, p_values, q_values, FORECAST_HORIZON, "ex7_fig2_gridsearch_heatmap.svg")

best_arima_name = f"ARIMA({best_p},1,{best_q})"
print(f"Best ARIMA from grid search: {best_arima_name}")

MODELS = {
    best_arima_name: ((best_p, 1, best_q), (0, 0, 0, 0)),
    "SARIMA(1,1,1)(1,1,1,24)": ((1, 1, 1), (1, 1, 1, 24)),
    "SARIMA(2,1,1)(0,1,1,24)": ((2, 1, 1), (0, 1, 1, 24)),
}

ARIMA(1,1,1)  MAE=0.2195  nRMSE=0.2483
ARIMA(1,1,2)  MAE=0.2194  nRMSE=0.2470
ARIMA(1,1,3)  MAE=0.2206  nRMSE=0.2506
ARIMA(1,1,4)  MAE=0.2204  nRMSE=0.2501
ARIMA(2,1,1)  MAE=0.2194  nRMSE=0.2472
ARIMA(2,1,2)  MAE=0.2197  nRMSE=0.2473


C:\Users\danyi\Desktop\TALTECH magister\Andmeteadus Energeetikas\hems\venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

C:\Users\danyi\Desktop\TALTECH magister\Andmeteadus Energeetikas\hems\venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.

C:\Users\danyi\Desktop\TALTECH magister\Andmeteadus Energeetikas\hems\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



ARIMA(2,1,3)  MAE=0.2205  nRMSE=0.2497


C:\Users\danyi\Desktop\TALTECH magister\Andmeteadus Energeetikas\hems\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\danyi\Desktop\TALTECH magister\Andmeteadus Energeetikas\hems\venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

C:\Users\danyi\Desktop\TALTECH magister\Andmeteadus Energeetikas\hems\venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.



ARIMA(2,1,4)  MAE=0.2206  nRMSE=0.2516
ARIMA(3,1,1)  MAE=0.2204  nRMSE=0.2506


...

C:\Users\danyi\Desktop\TALTECH magister\Andmeteadus Energeetikas\hems\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



ARIMA(3,1,3)  MAE=0.2217  nRMSE=0.2495


C:\Users\danyi\Desktop\TALTECH magister\Andmeteadus Energeetikas\hems\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



ARIMA(3,1,4)  MAE=0.2209  nRMSE=0.2514

Top ARIMA configurations:


Heatmap saved: images/ex7_fig2_gridsearch_heatmap.svg
Best ARIMA from grid search: ARIMA(1,1,2)


## 4(a) Single split validation based on the entire training dataset

In [11]:
metrics_single = []
for name, (order, seasonal) in MODELS.items():
    fitted = fit_arima(train, order, seasonal)
    fc = forecast_arima(fitted, FORECAST_HORIZON, test.index)
    m = evaluate_forecast(test.values, fc.values)
    m["model"] = name
    metrics_single.append(m)

metrics_single = pd.DataFrame(metrics_single)
print("\nSingle-split metrics:")
print(metrics_single[["model", "MAE", "RMSE", "nRMSE"]])

best_model = metrics_single.sort_values("nRMSE").iloc[0]["model"]
print(f"\nBest (single split): {best_model}")


Single-split metrics:
                     model       MAE      RMSE     nRMSE
0             ARIMA(1,1,2)  0.219427  0.291491  0.247026
1  SARIMA(1,1,1)(1,1,1,24)  0.139089  0.234173  0.198452
2  SARIMA(2,1,1)(0,1,1,24)  0.139403  0.233172  0.197603

Best (single split): SARIMA(2,1,1)(0,1,1,24)


## Forecast plot

In [12]:
best_order, best_seasonal = MODELS[best_model]
best_fit = fit_arima(train, best_order, best_seasonal)
best_fc = forecast_arima(best_fit, FORECAST_HORIZON, test.index)

fig = plot_forecast(
    timestamp=test.index,
    y_true=test,
    y_pred=best_fc,
    model_name=best_model,
    filename="ex7_fig3_forecast_overlay_best.svg"
)
fig.show()

## 4(b) Walk-forward validation (last 7 days)

In [13]:
wf_results = []
for name, (order, seasonal) in MODELS.items():
    _, met = walk_forward_daily(demand, order, seasonal)
    if met.empty:
        continue
    m = met[["MAE", "RMSE", "nRMSE"]].mean().to_dict()
    m["model"] = name
    wf_results.append(m)

wf_df = pd.DataFrame(wf_results)
print("\nWalk-forward metrics:")
print(wf_df)

C:\Users\danyi\Desktop\TALTECH magister\Andmeteadus Energeetikas\hems\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency h will be used.

C:\Users\danyi\Desktop\TALTECH magister\Andmeteadus Energeetikas\hems\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency h will be used.

C:\Users\danyi\Desktop\TALTECH magister\Andmeteadus Energeetikas\hems\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency h will be used.

C:\Users\danyi\Desktop\TALTECH magister\Andmeteadus Energeetikas\hems\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency h will be used.

C:\Users\danyi\Desktop\TALTECH magister\Andmeteadus Energeetikas\hems\venv\Lib\site-packages\statsmo


Walk-forward metrics:
        MAE      RMSE     nRMSE                    model
0  0.247385  0.336299  0.294529             ARIMA(1,1,2)
1  0.198293  0.306716  0.277173  SARIMA(1,1,1)(1,1,1,24)
2  0.200712  0.309297  0.281768  SARIMA(2,1,1)(0,1,1,24)


## 5 Models comparison

In [14]:
merged = pd.concat([
    metrics_single.assign(evaluation="Single-split"),
    wf_df.assign(evaluation="Walk-forward")
])

fig_bar = go.Figure()
for ev, df_ in merged.groupby("evaluation"):
    fig_bar.add_trace(go.Bar(x=df_["model"], y=df_["nRMSE"], name=ev))
fig_bar.update_layout(title="Model comparison (nRMSE)", barmode="group", **PLOT_STYLE)
save_fig_plotly(fig_bar, "ex7_fig4_model_comparison.svg")
fig_bar.show()

print("\nThe best overall:",
      wf_df.sort_values('nRMSE').iloc[0]['model'])



The best overall: SARIMA(1,1,1)(1,1,1,24)


### Which model performs better, and why?

SARIMA(1,1,1)(1,1,1,24) performed best overall.
It captures the daily seasonality of demand more effectively than ARIMA, leading to smaller prediction errors and better alignment with real hourly variations.

## Summary

Hourly demand data was analysed using ARIMA-family models.
ADF and KPSS tests showed mixed results, indicating seasonal non-stationarity.
To handle this, first-order differencing (`d=1`) and seasonal differencing (`D=1`, period=24) were applied.
ACF and PACF plots confirmed strong daily patterns.

Several ARIMA(p,1,q) models were tested, and the best non-seasonal model was ARIMA(1,1,2).
Two seasonal models, SARIMA(1,1,1)(1,1,1,24) and SARIMA(2,1,1)(0,1,1,24), were also trained and compared using single-split and walk-forward validation.
SARIMA models consistently achieved lower MAE and nRMSE values.

